In [54]:
#from __future__ import print_function
from scipy.misc import imsave
import numpy as np
import time
from keras import backend as K

from keras.models import model_from_json
import json

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [25]:
#load model
path = "/Users/augustus/python/CIFAR/models/vgg2_4.json"
json_object = open(path, "r")
loaded_json = json.loads(json_object.read())
json_object.close()

In [30]:
#load weights
model = model_from_json(loaded_json)
weights_path = "/Users/augustus/python/CIFAR/weights/vgg2_4_{val_acc:.2f}.hdf5"
model.load_weights(weights_path)